In [ ]:
score_Genentech<-function(dat,genes)
{
dat=log(dat+1)
genes=lapply(genes,function(x){return(intersect(x,rownames(dat)))})
print("Score gene sets")
res=lapply(genes,function(x){
rows=rownames(dat)
dat=data.frame(dat)
tab=dat[rows %in% x,]
print(length(x))
cols=colnames(dat)
tab=data.frame(t(tab))
for(i in colnames(tab)){tab[i]=tab[,i]-mean(tab[,i])}
mn=rowMeans(tab)
names(mn)=cols
return(mn)
})
ret=do.call(cbind,res)
colnames(ret)=names(genes)
return(ret)
}

In [ ]:
# Load in geneset lists and each replicate
library("tximport")
genes <- read.csv("/path/to/geneSetLists.csv")
filePath <- "/path/to/data/REGION/"
sampleNames <- c()
files <- file.path(filePath, paste0(sampleNames, "/", sampleNames, ".genes.results"))
names(files) <- sampleNames
rsem.in <- tximport(files, type="rsem", txIn=FALSE, txOut=FALSE)
dat <- rsem.in$abundance

geneName <- data.frame(do.call('rbind', strsplit(as.character(row.names(dat)), '_', fixed=TRUE)))
geneName <- geneName["X2"]               
row.names(dat) <- geneName$X2

head(dat)
colSums(dat)

In [ ]:
# Run scoring function
res <- score_Genentech(dat, genes)
row.names(res) <- colnames(dat)
head(res)
# Transpose output table for readability (optional)
res2 <- t(res)
write.csv(res2, "EXPERIMENT_geneSetScoresByReplicate.csv")

In [ ]:
# Reformat results and add genotype information 
res <- as.data.frame(res)
condition <- c(rep("GENO1",W),rep("GENO2",X),rep("GENO3",Y),rep("WT",Z))
res$condition <- condition
res

# Isolate results from GENO1 + WT for comparison
res_GENO1 <-  res[res$condition %in% c("GENO1","WT"),]
GENO1_names <- c()
idx_GENO1 <- match(GENO1_names, names(dat))
dat_GENO1 <- dat[,idx_GENO1]
# Isolate results from GENO2 + WT for comparison
res_GENO2 <- res[res$condition %in% c("GENO2","WT"),]
GENO2_names <- c()
idx_GENO2 <- match(GENO2_names, names(dat))
dat_GENO2 <- dat[,idx_GENO2]

# Isolate results from GENO3 + WT for comparison
res_GENO3 <- res[res$condition %in% c("GENO3","WT"),]
GENO3_names <- c()
idx_GENO3 <- match(GENO3_names, names(dat))
dat_GENO3 <- dat[,idx_GENO3]

In [ ]:
# Fit linear model for all pathways in GENO1
    # iterate over each pathway column and run linear model on each
    # write output to mergeoutGENO1
mergeOutGENO1 <- data.frame(matrix(ncol=4,nrow=0))
colnames(mergeOutGENO1) <- c("Estimate", "Std. Error", "t value", "p_value")
for(i in names(res_GENO1[,1:16])){
    model = lm(res_GENO1[,i]~res_GENO1$condition,data=as.data.frame(dat_GENO1))
    mergeOutGENO1[i,] <- as.data.frame(t(summary(model)$coefficients[2,]))
}
mergeOutGENO1
write.csv(mergeOutGENO1, "EXPERIMENT_GENO1_geneSetStats.csv", row.names=FALSE)

In [ ]:
# Fit linear model for all pathways in GENO2
mergeOutGENO2 <- data.frame(matrix(ncol=4,nrow=0))
colnames(mergeOutGENO2) <- c("Estimate", "Std. Error", "t value", "p_value")
for(i in names(res_GENO2[,1:16])){
    model = lm(res_GENO2[,i]~res_GENO2$condition,data=as.data.frame(dat_GENO2))
    mergeOutGENO2[i,] <- as.data.frame(t(summary(model)$coefficients[2,]))
}
mergeOutGENO2
write.csv(mergeOutGENO2, "EXPERIMENT_GENO2_geneSetStats.csv", row.names=FALSE)

In [ ]:
# Fit linear model for all pathways in GENO3
mergeOutGENO3 <- data.frame(matrix(ncol=4,nrow=0))
colnames(mergeOutGENO3) <- c("Estimate", "Std. Error", "t value", "p_value")
for(i in names(res_GENO3[,1:16])){
    model = lm(res_GENO3[,i]~res_GENO3$condition,data=as.data.frame(dat_GENO3))
    mergeOutGENO3[i,] <- as.data.frame(t(summary(model)$coefficients[2,]))
}
mergeOutGENO3
write.csv(mergeOutGENO3, "EXPERIMENT_GENO3_geneSetStats.csv", row.names=FALSE)